In [1]:
import pandas as pd
import numpy as np
import aux.acessos as ac
import aux.perfil as pf

In [2]:
query_at = """with aut as (
select	
nr_cpf as cpf,
substring(cast(dt_autorizacao as varchar), 1, 10) as dia,
sum(vl_real) as spending_aut 
from	platform_curated_zone.authorization_will
where   ds_transacao = 'credito'	
and	ds_status_compra = 'aprovada'
and ds_mcc != 'TRANSACOES WEBSERVICE'
and	cd_retorno = '00'
and ds_customer = 'will'
and substring(cast(dt_autorizacao as varchar), 1, 10) in ('2021-10-08','2021-11-10','2021-12-20', '2022-01-15', '2022-02-16')
group by 1,2
)
, trans as (
select  
cpf,
substring(cast(dt_autorizacao as varchar), 1, 10) as dia,
sum(vl_transacao) as spending_trans
from backoffice_curated_zone.transaction
where substring(cast(dt_autorizacao as varchar), 1, 10) in ('2021-10-08','2021-11-10','2021-12-20', '2022-01-15', '2022-02-16')
and is_spending='Sim' --spending de credito retirando saque e recarga
and is_cancelamento = 'Nao'	
and lower(ds_empresa) = 'will'
group by 1,2 
)
select 
a.cpf, b.cpf as cpf_trans,
a.dia, b.dia as dia_trans,
spending_trans,
spending_aut
from aut a
full join trans b on (b.dia = a.dia and b.cpf = a.cpf) """

df_at = ac.df_athena('flavia-costa', query_at)

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [3]:
df_at.head()

,cpf,cpf_trans,dia,dia_trans,spending_trans,spending_aut
0,05384581190,05384581190,2021-12-20,2021-12-20,10.20,10.20
1,00558198503,00558198503,2021-12-20,2021-12-20,12.10,12.10
2,12604357682,12604357682,2021-12-20,2021-12-20,181.44,181.44
3,13832538704,13832538704,2021-12-20,2021-12-20,30.65,30.65
4,14550166673,None,2021-12-20,None,NaN,126.19


In [4]:
len(df_at.query('spending_aut > spending_trans'))

7438

In [9]:
len(df_at.query('spending_aut != spending_trans'))

9575

In [5]:
len(df_at)

51322

In [6]:
df_at['dif'] = df_at['spending_aut'] - df_at['spending_trans']

In [7]:
df_at['dif'].mean()

10.458368523237473

In [18]:
len(df_at.query('dif > 1 or dif < -1'))

2631

In [33]:
agg = df_at.query('dif > 1 or dif < -1').groupby('dia').agg({'dif':['mean']} )

In [34]:
agg.head()

,dif
,mean
dia,
2021-10-08,205.158029
2021-11-10,268.409362
2021-12-20,274.350027
2022-01-15,117.154031
2022-02-16,203.852652


In [22]:
df_at.query('dif > 1 or dif < -1').sample(5)

,cpf,cpf_trans,dia,dia_trans,spending_trans,spending_aut,dif
14436,73469432104,73469432104,2022-02-16,2022-02-16,2630.11,2700.69,70.58
34527,00536157600,00536157600,2022-02-16,2022-02-16,750.00,3012.96,2262.96
42534,19132829701,19132829701,2022-02-16,2022-02-16,63.69,70.76,7.07
17955,13537900708,13537900708,2022-01-15,2022-01-15,11.61,22.79,11.18
7075,10993392431,10993392431,2022-02-16,2022-02-16,653.54,668.43,14.89
